### Load libraries and enviromental variables

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from dotenv import load_dotenv
import pandas as pd
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import numpy as np
import neptune
from neptune.utils import stringify_unsupported
from datasets import load_metric

print(load_dotenv(dotenv_path='../'))
#from custom_utils.custom_pytorch_utils import TranslationDataset, collate_fn, evaluate_model_on_metrics

False


### Load model and setup device

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSeq2SeqLM.from_pretrained("model/model")
tokenizer = AutoTokenizer.from_pretrained("model/tokenizer/")
print(device)
model.to(device);

cuda


### Load metrics

In [3]:
bleu_metric = load_metric("bleu", trust_remote_code=True)
meteor_metric = load_metric("meteor", trust_remote_code=True)
rouge_metric = load_metric("rouge", trust_remote_code=True)

C:\Users\urbii\AppData\Local\Temp\ipykernel_41192\1174947439.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu", trust_remote_code=True)
Using the latest cached version of the module from C:\Users\urbii\.cache\huggingface\modules\datasets_modules\metrics\bleu\05eb00cc2401cfd07edf1c3b09cdcbee37d65e58d5dd07d9c3e0ee3725ddf7e4 (last modified on Thu Aug  1 13:11:03 2024) since it couldn't be found locally at bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\urbii\.cache\huggingface\modules\datasets_modules\metrics\meteor\66efa51c366195fce5b82fcd16a5295716fff778276b529048fed7ca742ec3d3 (last modified on Thu Aug  1 13:11:05 2024) since it couldn't be found locally at meteor, or remotely on the Hugging Face Hub.
[nltk_data] Error loading wordnet: <urlopen 

### Load and prepare data

In [4]:
data = pd.read_csv("data/All_data.csv")
train_indices = data.sample(frac=0.85, random_state=42).index
train_data = data.loc[train_indices].reset_index(drop=True)
valid_data = data.drop(train_indices).reset_index(drop=True)

print(train_data.shape)
print(valid_data.shape)

train_data.head()

(1029, 2)
(182, 2)


,pl,mig
0,Jaki ma Pan numer domu?,Jaki twój dom numer
1,Czy ma Pani krewnych?,Ty krewni masz
2,Jak nazywa się prezydent polski?,Prezydent polska nazwisko
3,Jakie Pani ma obywatelstwo?,Ty obywatelstwo jakie
4,Gdzie mieszkasz?,Ty mieszkać gdzie


In [5]:
train_dataset = TranslationDataset(train_data.pl, train_data.mig, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

valid_dataset = TranslationDataset(train_data.pl, train_data.mig, tokenizer)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
next(iter(train_dataloader))

{'input_ids': tensor([[   12,   243,  9805,    61,    51,   142,  1247,  1124, 14288,     7,
              0],
         [  883,    45, 39906,     7,     0, 63429, 63429, 63429, 63429, 63429,
          63429],
         [ 5494, 17812,  1647,  1184,     2,     0, 63429, 63429, 63429, 63429,
          63429],
         [  322,   487,   363,  3032,   172,     2,     0, 63429, 63429, 63429,
          63429],
         [ 3049,  2320,   140, 11656,    26, 31941,     7,     0, 63429, 63429,
          63429],
         [  322,    26,  2586,    25,    83,    24,  1171,     2,     0, 63429,
          63429],
         [  116,  4878,   485, 10360,   490,    24,   432,    31,  1524,     2,
              0],
         [  362,   606,  7549,  1450,   111,   466,   461,  6252,    61,     7,
              0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       

### Model training

In [ ]:
run = neptune.init_run()
lr = 5e-5
num_epochs = 5
optimizer = Adam(model.parameters(), lr=lr)

run["hyperparameters/learning_rate"] = optimizer.state_dict()['param_groups'][0]['lr']
run["hyperparameters/optimizer"] = "Adam"
run["hyperparameters/betas"] = stringify_unsupported(optimizer.state_dict()['param_groups'][0]['betas'])
run["hyperparameters/eps"] = optimizer.state_dict()['param_groups'][0]['eps']
run["datasets/train"].track_files("data/All_data.csv")
run["hyperparameters/num_epochs"] = num_epochs

model.train()
for epoch in range(num_epochs):
    loss_all = 0
    for batch in tqdm(train_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_all += loss.item()
    run["train/loss"].append(np.round(loss_all / len(train_dataloader), 4))
    print(f"Epoch: {epoch + 1}, loss: {np.round(loss_all / len(train_dataloader), 4)}")
    
run["score/final_loss"] = np.round(loss_all / len(train_dataloader), 4)

100%|██████████| 129/129 [00:30<00:00,  4.16it/s]


Epoch: 1, loss: 3.0501


100%|██████████| 129/129 [00:29<00:00,  4.40it/s]


Epoch: 2, loss: 1.3466


100%|██████████| 129/129 [00:29<00:00,  4.39it/s]


Epoch: 3, loss: 0.8305


100%|██████████| 129/129 [00:29<00:00,  4.44it/s]


Epoch: 4, loss: 0.5876


100%|██████████| 129/129 [00:29<00:00,  4.42it/s]

Epoch: 5, loss: 0.44


In [ ]:
bleu_score, meteor_score, rouge_score = evaluate_model_on_metrics(model, valid_dataloader, tokenizer, bleu_metric, meteor_metric, rouge_metric, device)
run["metrics/BLEU"] = bleu_score
run["metrics/METEOR"] = meteor_score
run["metrics/ROUQE"] = rouge_score
run.stop()

In [8]:
print(f"BLEU: {bleu_score} METEOR: {meteor_score}, ROUGE: {rouge_score}")

BLEU: 0.704 METEOR: 0.788, ROUGE: 0.841
